# Code Review Team mit Semantic Kernel
## Erstellen von spezialisierten Agenten für Code-Überprüfung

### 1. Einführung in SK Agents
Agents in Semantic Kernel sind KI-gesteuerte Entitäten, die in der Lage sind, Gespräche zu führen, Entscheidungen zu treffen und Aufgaben auszuführen. Sie können unabhängig arbeiten oder in Gruppen zusammenarbeiten, um komplexe Ziele zu erreichen.

### 2. Erstellen von spezialisierten Agenten

In [4]:
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from pydantic import BaseModel
from typing import List

# Definieren der spezialisierten Agenten
class CodeInspector:
    @kernel_function(description="Analyze code for bugs")
    def analyze_code(self, code: str) -> str:
        # Implementierung
        pass

class StyleReviewer:
    @kernel_function(description="Check coding style and conventions")
    def check_style(self, code: str) -> str:
        # Implementierung
        pass

class SecurityAuditor:
    @kernel_function(description="Identify security vulnerabilities")
    def audit_security(self, code: str) -> str:
        # Implementierung
        pass

class AutoFixer:
    @kernel_function(description="Suggest code improvements")
    def suggest_fixes(self, code: str) -> str:
        # Implementierung
        pass

### 3. Implementierung der Gruppenchat-Funktionalität

In [5]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

class ApprovalTerminationStrategy(TerminationStrategy):
    async def should_agent_terminate(self, agent, history):
        return "approved" in history[-1].content.lower()

# Initialisieren des Kernels und der Agenten
kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="code_review"))

# Hinzufügen der Plugins zum Kernel
kernel.add_plugin(CodeInspector(), plugin_name="code_inspector")
kernel.add_plugin(StyleReviewer(), plugin_name="style_reviewer")
kernel.add_plugin(SecurityAuditor(), plugin_name="security_auditor")
kernel.add_plugin(AutoFixer(), plugin_name="auto_fixer")

# Erstellen der Agenten mit Zugriff auf die Plugins
agent_inspector = ChatCompletionAgent(
    service_id="code_review",
    kernel=kernel,
    name="CodeInspector",
    instructions="Analyze code for bugs and issues."
)

agent_style = ChatCompletionAgent(
    service_id="code_review",
    kernel=kernel,
    name="StyleReviewer",
    instructions="Check coding style and conventions."
)

agent_security = ChatCompletionAgent(
    service_id="code_review",
    kernel=kernel,
    name="SecurityAuditor",
    instructions="Identify security vulnerabilities."
)

agent_auto_fixer = ChatCompletionAgent(
    service_id="code_review",
    kernel=kernel,
    name="AutoFixer",
    instructions="Suggest code improvements."
)

# Definieren des Gruppenchats mit Terminationsstrategie
chat = AgentGroupChat(
    agents=[agent_inspector, agent_style, agent_security, agent_auto_fixer],
    termination_strategy=ApprovalTerminationStrategy(
        agents=[agent_inspector, agent_style, agent_security], maximum_iterations=10
    ),
)


### 4. Rückgabe der strukturierten Ergebnisse

In [6]:
class CodeReviewResult(BaseModel):
    bugs_found: List[str]
    style_issues: List[str]
    security_concerns: List[str]
    approval_status: str  # "approved", "needs_work", "rejected"
    recommended_fixes: List[str]


### 5. Beispiel-Code-Snippets zur Überprüfung

In [7]:
code_snippets = [
    "def buggy_function(x): return x / 0",  # Einfache Funktion mit Fehlern
    "def style_issue_function(x):return x*2",  # Code mit Stilproblemen
    "def security_vulnerable_function(password): print(password)"  # Code mit Sicherheitslücken
]

# Benutzer initiiert das Gespräch
for code in code_snippets:
    user_input = f"Review the following code:\n{code}"
    await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))
    print(f"# {AuthorRole.USER}: '{user_input}'")

    # Verarbeiten der Antworten der Agenten
    bugs_found = []
    style_issues = []
    security_concerns = []
    recommended_fixes = []
    approval_status = "needs_work"

    async for response in chat.invoke():
        print(f"# {response.role} - {response.name or '*'}: '{response.content}'")
        if response.name == "CodeInspector":
            bugs_found.append(response.content)
        elif response.name == "StyleReviewer":
            style_issues.append(response.content)
        elif response.name == "SecurityAuditor":
            security_concerns.append(response.content)
        elif response.name == "AutoFixer":
            recommended_fixes.append(response.content)

    print(f"# Chat Complete: {chat.is_complete}")

    # Rückgabe der strukturierten Ergebnisse
    review_result = CodeReviewResult(
        bugs_found=bugs_found,
        style_issues=style_issues,
        security_concerns=security_concerns,
        approval_status=approval_status,
        recommended_fixes=recommended_fixes
    )
    print(review_result)

# AuthorRole.USER: 'Review the following code:
def buggy_function(x): return x / 0'


# AuthorRole.ASSISTANT - CodeInspector: 'The provided code has a significant issue:

1. **Division by Zero**: The function attempts to divide `x` by `0`, which will lead to a `ZeroDivisionError` in Python. This is a runtime error and will cause the program to crash if this function is called with any value of `x`.

Here’s a refined version of the function that handles the division by zero error:

```python
def safe_function(x):
    try:
        return x / 0  # This line will still raise an error
    except ZeroDivisionError:
        return "Cannot divide by zero"
```

In this updated code, if the function encounters a division by zero, it will return a friendly message instead of crashing the program. You may want to alter the logic depending on the intended functionality of the function.'
# AuthorRole.ASSISTANT - StyleReviewer: 'The provided code has a significant issue:

1. **Division by Zero**: The function attempts to divide `x` by `0`, which will lead to a `ZeroDivisionError` in P